Pull dataforest and install

In [ ]:
pip install -e ../dataforest

In [ ]:
pip install -e .

In [ ]:
# install tree (filesystem viewer used later)
!apt-get install tree

In [ ]:
%load_ext autoreload
%autoreload 2

import cellforest
from cellforest import Counts
import pandas as pd
from pathlib import Path

# Load Sample Data

In [ ]:
cellranger_dir = Path("tests/data/v3_gz/sample_1")

In [ ]:
from tests.utils.get_test_data import get_test_data
if not cellranger_dir.exists():
    get_test_data()
    if not cellrangder_dir.exists():
        raise ValueError("Notebook must be updated to conform to `get_test_data`")

In [ ]:
ls {cellranger_dir}

# Quick Start

## Specify root of working directory tree

Use any directory, doesn't have to exist yet

In [ ]:
example_dir = "tests/data/example_usage"

In [ ]:
root_dir = f"{example_dir}/root"

# Counts Matrix

The counts matrix is a cells x genes matrix, built as a wrapper around `scipy.sparse.csr_matrix`. 

This data structure is central to the functionality of cellforest, so it's important to understand how it works. When using cellforest, you normally won't load/save it directly, but rather let cellforest handle that. However, if you don't need to automate workflows, and just want to do some counts matrix analysis outside of cellforest, you may want to instantiate it directly.

### Load from cellranger

`Counts` objects can be instantiated directly from cellranger outputs

In [ ]:
rna = Counts.from_cellranger(cellranger_dir)

In [ ]:
rna

### Data attributes

The sparse matrix is stored in the `_matrix` attribute, which you generally shouldn't interact with directly. The `Counts` object has inherited most of the relevant methods of `csr_matrix`, so you can still do relevant calculations

In [ ]:
rna.shape

In [ ]:
rna.sum()

The genes and ensembl IDs are stored in `features`, and can also be accessed via `genes` and `ensgs`, respectively. These function as the column index for the matrix. Genes can also be accessed via `columns` (like pandas). Note that the enseml IDs will be stripped in any data downstream of a Seurat process, since Seurat lacks ensembl support.

In [ ]:
rna.features.head()

The 10X cell barcodes are stored in `cell_ids`, which can also be accessed via `index` (like pandas)

In [ ]:
rna.cell_ids.head()

### Slicing

The matrix can be sliced with integer indices, cell_ids, gene names, or ensembl IDs. The latter three can be presented in the form of strings, lists of strings, or `pandas.Series`. The matrix, features, and cell_ids will all be sliced correspondingly.

In [ ]:
rna[:10, :20]

In [ ]:
rna["AAACATACAACCAC-1"]

In [ ]:
rna[["AAACATACAACCAC-1", "AAACATTGATCAGC-1"]]

In [ ]:
rna[["AAACATACAACCAC-1", "AAACATTGATCAGC-1"], 1:10]

In [ ]:
rna[:, ["MIR1302-10", "RP11-34P13.7"]]

In [ ]:
rna[:, "ENSG00000238009"]

In [ ]:
rna[rna.cell_ids[:20]]

### Plotting

In [ ]:
# TODO: histogram notimplemented

In [ ]:
# TODO: scatter notimplemented

### Concatenation

A list of `Counts` objects can be `concatenate`ed, or one or more `Counts` objects can be appended to an existing one. Concatenation can occur along either the cells (`axis=0`) or genes (`axis=1`) dimensions, whereas `append` assumes the cells dimension. `hstack` and `vstack` can be used as alternatives to concatenation along the cells and genes axes, respectively (like numpy).

In [ ]:
rna_2 = Counts.concatenate([rna[:20], rna[30:50]])
rna_2

In [ ]:
Counts.concatenate([rna[:, :20], rna[:, 30:50]], axis=1)

In [ ]:
rna_2.append(rna[60:100])

`append` is not an `inplace` operation

In [ ]:
rna_2

### Drop

We can drop specified cells or genes. This doesn't occur `inplace`

In [ ]:
# TODO: .drop not implemented

We can also drop cells or genes with no UMIs

In [ ]:
rna.dropna()

In [ ]:
rna.dropna(axis=1)

### I/O

**DataFrame (converts to dense)**

In [ ]:
rna.to_df().head()

**Save**

In [ ]:
example_counts_path = "tests/data/example/counts/rna.pickle"
rna.save(example_counts_path)

In [ ]:
ls {example_counts_path}

**Load**

In [ ]:
Counts.load(example_counts_path)

**Others**

There are also `to_cellranger`, `to_rds`, and `from_rds` I/O methods. Note that they may leave behind intermediate artifacts (e.g. pickle files).

# Cellforest Loading Data

We can load data from cellranger outputs. If there are multiple samples and metadata is available, option 3 should be used. The data is loaded and combined in a `Counts` matrix as an attribute of our `CellForest` object. Python (.pickle) and Seurat (.rds) versions are saved in our `root_dir`. A `meta.tsv` file will also be created, which will include `cell_id`s (barcodes) as an index, and any additional sample metadata for each cell.

The `root_dir` will serve as the base for all of our downstream analysis. Once this directory has been populated, you can use option 4 to load from the .pickle file rather than re-processing the cellranger outputs.

### Option 1: from single cellranger output

In [ ]:
cf = cellforest.from_input_dirs(root_dir, cellranger_dir)
cf

In [ ]:
ls {root_dir}

In [ ]:
pd.read_csv(f"{root_dir}/meta.tsv", sep="\t", index_col=0).head()

### Option 2: from multiple cellranger outputs

In [ ]:
cellranger_dir_2 = "tests/data/v3_gz/sample_2"
cf = cellforest.from_input_dirs(root_dir, [cellranger_dir, cellranger_dir_2])

### Option 3 (PREFERRED): From metadata

This is preferred because this will allow you to include your metadata in analysis

In [ ]:
# load example metadata
meta = pd.read_csv("tests/data/sample_metadata.tsv", sep="\t")
meta.head()

In [ ]:
cf = cellforest.from_metadata(root_dir, meta)

### Option 4 (for every subsequent load): From existing root

In [ ]:
cf = cellforest.load(root_dir)

# Cellforest Interface

## Metadata

In [ ]:
cf.meta

## Counts

In [ ]:
cf.rna

Other useful methods include `copy`, 

## Workflow automation

The purpose of cellforest isn't just to interact with metadata and counts matrices -- we want to automate workflows and interact with the outputs. We do this with a specification, which we input as a dictionary, and gets converted to a `Spec` object internally. Each key represents a process name, and the values represent input parameters to that process.

In [ ]:
spec = [
    {
        "process": "normalize",
        "params": {
            "min_genes": 5,
            "max_genes": 5000,
            "min_cells": 5,
            "nfeatures": 30,
            "perc_mito_cutoff": 20,
            "method": "seurat_default",
        }
]

In [ ]:
cf = cellforest.from_metadata(root_dir, meta, spec=spec)

In [ ]:
ls {root_dir}

We can now execute processes from the spec

In [ ]:
cf.process.normalize()

In [ ]:
ls {root_dir}

In [ ]:
!tree {root_dir}

The catalogue uses the `run_spec` to look up the `run_id` from the `run_catalogue` if one exists already, and stores one if not.

In [ ]:
pd.read_csv(Path(root_dir) / "normalize/run_catalogue.tsv", sep="\t", index_col="run_spec")

# Process

The default processes are defined in `cellforest.process`

In [ ]:
from cellforest.processes import processes
process_path = str(Path(processes.__file__).parent)

In [ ]:
!tree {process_path}

The `process.py` file contains the process function, which should be decorated with `dataprocess` (explained in Hooks section). `normalize/process.py` is pasted below.

The function will be broken up below

In [ ]:
from dataforest.hooks import dataprocess

from cellforest.utils.r.run_r_script import run_process_r_script


@dataprocess(requires="root", matrix_layer=True)
def normalize(forest: "CellForest"):
    process_name = "normalize"
    input_metadata_path = forest.get_temp_meta_path(process_name)
    # TODO: add a root filepaths lookup
    input_rds_path = forest.root_dir / "rna.rds"
    output_rds_path = forest[process_name].path_map["rna_r"]
    min_genes = forest.spec[process_name]["min_genes"]
    max_genes = forest.spec[process_name]["max_genes"]
    min_cells = forest.spec[process_name]["min_cells"]
    perc_mito_cutoff = forest.spec[process_name]["perc_mito_cutoff"]
    r_functions_filepath = forest.schema.__class__.R_FILEPATHS["FUNCTIONS_FILE_PATH"]
    method = forest.spec[process_name]["method"]
    arg_list = [
        input_metadata_path,
        input_rds_path,
        output_rds_path,
        min_genes,
        max_genes,
        min_cells,
        perc_mito_cutoff,
        r_functions_filepath,
    ]
    if method == "sctransform":
        output_corrected_umi_path = forest[process_name].path_map["corrected_umi"]
        output_pearson_residual_path = forest[process_name].path_map["pearson_residual"]
        arg_list += [output_corrected_umi_path, output_pearson_residual_path]
        r_normalize_script = str(forest.schema.__class__.R_FILEPATHS["SCTRANSFORM_SCRIPT"])
    elif method == "seurat_default":
        verbose = True
        verbose = str(verbose).upper()
        nfeatures = forest.spec[process_name]["nfeatures"]
        arg_list += [verbose, nfeatures]
        r_normalize_script = str(forest.schema.__class__.R_FILEPATHS["SEURAT_DEFAULT_NORMALIZE_SCRIPT"])
    else:
        raise ValueError(f"Invalid normalization method: {method}. Use 'sctransform' or 'seurat_default'")
    run_process_r_script(forest, r_normalize_script, arg_list, process_name)


`forest` simply refers to the `CellForest` object (`cf`) upon which we're executing this process
A hook function runs prior to the process start, which stores temporary metadata, which is removed afterward (more in hooks section). Here we're getting the path to that metadata

In [ ]:
def normalize(forest: "CellForest"):
    process_name = "normalize"
    input_metadata_path = forest.get_temp_meta_path(process_name)

The second line below uses the `forest[process_name]` syntax, which refers to a `ProcessRun` object.

In [ ]:
    input_rds_path = forest.root_dir / "rna.rds"
    output_rds_path = forest[process_name].path_map["rna_r"]

In [ ]:
pr = cf["normalize"]
pr

The process run has a few attributes to navigate the file structure

File names

In [ ]:
pr.file_map

Path to output directory

In [ ]:
pr.path

Full paths.

In [ ]:
pr.path_map

There are two more attributes `files` and `filepaths`, which are just lists of the keys of `file_map` and `path_map`, respectively.

Check if the process has been run (just checks for logfiles in process run directory)

In [ ]:
pr.done

Check output files

In [ ]:
ls {str(pr.path)}

Check log files (`normalize.out` and `normalize.err`). They are intented to be stdout and stderr, but in reality, lines written in cellforest R code go to .out, and lines from other packages, e.g. Seurat, go to .err.

In [ ]:
pr.logs

Back to the process definition: these lines are simply extracting the parameters from the spec.

In [ ]:
    min_genes = forest.spec[process_name]["min_genes"]
    max_genes = forest.spec[process_name]["max_genes"]
    min_cells = forest.spec[process_name]["min_cells"]
    perc_mito_cutoff = forest.spec[process_name]["perc_mito_cutoff"]
    method = forest.spec[process_name]["method"]

We can check out the spec to see those values

In [ ]:
cf.spec

This line gets the filepath to some R utility functions from our configuration. It accesses the `__class__` attribute due to a metaprogramming implementation detail that allows us to dynamically change the configuration.

In [ ]:
    r_functions_filepath = forest.schema.__class__.R_FILEPATHS["FUNCTIONS_FILE_PATH"]

To inspect the configuration:

In [ ]:
# TODO: MAKE DATAHOOK - E.G. TO USE LATEST MATRIX, OR TO CONCAT CLUSTERS TO META

To update the configuration

Generally, you wouldn't want to dynamically update the configuration like this, but rather pass it in when you instantiate your `CellForest`. 

What about having two of them with different configs? Test this -- if a problem, use a more global feeling interface to update it

The normalize process both filters out cells by mitochondrial fraction and does counts matrix normalization using either Seurat default normalization or sctransform, as specified in the parameters. It outputs 

### Data Specification

In addition to parameter specification, we may also want to specify the data which flows into each process. We can do that either by subsetting the data, to include only those which match the specification, or by filtering the data to exclude those which match the specification.

**Subset**

### Filter

### Partition

# Customizing modules